### Classification of ag plastics in Oxnard, CA

Annie Taylor, June 2024

Sentinel-2 classification of agricultural plastic use in Oxnard, CA based on Yuanyuan Tian's summer 2023 project.

Converted from the GEE javascript to the GEE python api

#### Import packages and initialize ee api

In [1]:
import numpy as np
import pandas as pd
import ee
import geemap

In [2]:
# ee.Authenticate(auth_mode='localhost')

In [3]:
# ee.Initialize()
ee.Initialize(project='ee-annalisertaylor')

#### Import ee features

In [4]:
assets = 'projects/ee-annalisertaylor/assets/TNC/agplastics/'

oxnard_bnd = ee.FeatureCollection(assets + 'Oxnard')
strawberries = ee.FeatureCollection(assets + 'Strawberries')
bushberries = ee.FeatureCollection(assets + 'Bushberries_T19')
flowers = ee.FeatureCollection(assets + 'Flowers_T16')
peppers = ee.FeatureCollection(assets + 'Peppers_T21')
avocados = ee.FeatureCollection(assets + 'Avocados')
label_mulch_hoop = ee.FeatureCollection(assets + 'label_mulch_hoop')
label_nonplastic = ee.FeatureCollection(assets + 'label_nonplastic')
polygons = label_mulch_hoop.merge(label_nonplastic)

aoi_truck_crop = ee.FeatureCollection(assets + 'T')
aoi = aoi_truck_crop

In [5]:
# # importing shapefiles locally and converting to ee features was super slow and causing timeout

# oxnard_bnd = geemap.shp_to_ee('data/agplastics/Oxnard.shp') 
# strawberries = geemap.shp_to_ee('data/agplastics/Strawberries.shp')
# bushberries = geemap.shp_to_ee('data/agplastics/Bushberries_T19.shp')
# flowers = geemap.shp_to_ee('data/agplastics/Flowers_T16.shp')
# peppers = geemap.shp_to_ee('data/agplastics/Pepers_T21.shp')
# avocados = geemap.shp_to_ee('data/agplastics/Avocados.shp')
# label_mulch_hoop = geemap.shp_to_ee('data/agplastics/label_mulch_hoop.shp')
# label_nonplastic = geemap.shp_to_ee('data/agplastics/label_nonplastic.shp')
# polygons = label_mulch_hoop.merge(label_nonplastic)

# aoi_truck_crop = geemap.shp_to_ee('data/agplastics/T.shp')
# aoi = aoi_truck_crop

#### Date range

In [6]:
start = '2019-02-01'
end = '2019-06-01'

In [7]:
m = geemap.Map()
m.add("basemap_selector")
m.addLayer(oxnard_bnd)
m.centerObject(oxnard_bnd, 12)
m

Map(center=[34.19948528797552, -119.1495747099073], controls=(WidgetControl(options=['position', 'transparent_…

#### Sentinel-2 data processing

##### helper functions

In [8]:
# mask out cloudy pixels from s2 images
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Add NDVI band
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Add NDTI band
def addNDTI(image):
    ndti = image.normalizedDifference(['B11', 'B12']).rename('NDTI')
    return image.addBands(ndti)

# Add PGI band
def addPGI(image):
    NIR = image.select('B8')
    R = image.select('B4')
    G = image.select('B3')
    B = image.select('B2')
    pgi = NIR.subtract(R).multiply(100).multiply(R) \
      .divide(NIR.add(B).add(G).divide(3).subtract(1)) \
      .rename('PGI')
    return image.addBands(pgi)

# Add PMLI band
def addPMLI(image):
    SWIR1 = image.select('B11')
    R = image.select('B4')
    pmli = SWIR1.subtract(R) \
      .divide(SWIR1.add(R)) \
      .rename('PMLI')
    return image.addBands(pmli)

# Add RPGI band
def addRPGI(image):
    NIR = image.select('B8')
    R = image.select('B4')
    G = image.select('B3')
    B = image.select('B2')
    rpgi = B.multiply(100) \
      .divide(NIR.add(B).add(G).divide(3).subtract(1)) \
      .rename('RPGI')
    return image.addBands(rpgi)



##### build the collection

In [9]:
processedCollection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate(start, end) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .map(addNDVI) \
    .map(addNDTI) \
    .map(addPGI) \
    .map(addPMLI) \
    .map(addRPGI)

In [10]:
# Get the median image from the processed collection and clip it to the AOI
# This is the image that the model is trained on
image = processedCollection.median().clip(aoi).clip(oxnard_bnd);

# Select which bands to use for plastic presence prediction
# ref: Combining Multi-Source Data and Feature Optimization for Plastic-Covered Greenhouse Extraction
#     and Mapping Using the Google Earth Engine : A Case in Central Yunnan
bands = ['B4', 'B3', 'B2', 'B6', 'B8', 'B11', 'B12', 'NDVI', 'NDTI', 'PGI', 'PMLI']



#### Prepare training data

In [11]:
# Get the values for all pixels in each polygon, takes 4 min to run even with only one band selected :/ 
# Changing this so that it's only sampling one band, do we need every point within these polygons?
data = image.select('B4').sampleRegions(**{
  'collection': polygons,
  'properties': ['class'],
  'scale': 30, #30m, TODO why is this based on Landsat scale? EE uses nearest neighbor to downsample
  'geometries': True
})
print('total number of labeled data points: ', data.size().getInfo())

total number of labeled data points:  2125


In [13]:
#-------------------- process data (1): split---------------------
# The randomColumn() method will add a column of uniform random
# numbers in a column named 'random' by default.
data = data.randomColumn(seed=0)

split = 0.7;  # Roughly 70% training, 30% testing.
training = data.filter(ee.Filter.lt('random', split))
validation = data.filter(ee.Filter.gte('random', split))

# print('number of training points: ', training.size().getInfo())
# print('number of validation points: ', validation.size().getInfo())

In [16]:
#-------------------- process data (2): address autocorrelation correction---------------------
# Spatial join.
distFilter = ee.Filter.withinDistance(**{
  'distance': 30, # unit: meter. 
  'leftField': '.geo',
  'rightField': '.geo',
  'maxError': 10
})

join = ee.Join.inverted()

# Apply the join
training = join.apply(training, validation, distFilter)
print('training data points after removing autocorrelated points: ', training.size().getInfo())


training after removing autocorrelated points:  444


#### Train the models

In [17]:
#-----------model 1: SVM-----------#
# Create an SVM classifier with custom parameters.
classifier_SVM = ee.Classifier.libsvm(**{
  'kernelType': 'RBF',
  'gamma': 0.5,
  'cost': 10
})

# Train the classifier.
trained_SVM = classifier_SVM.train(training, 'class', bands)

# Classify the image.
classified_SVM = image.classify(trained_SVM)

# #-----------model 2: CART-----------#
# # Train a CART classifier with default parameters.
# classifier_CART = ee.Classifier.smileCart(**{
# })

# # Train the classifier.
# trained_CART = classifier_CART.train(training, 'class', bands)

# # Classify the image with the same bands used for training.
# classified_CART = image.select(bands).classify(trained_CART)

# #-----------model 3: RF-----------#
# # Make a Random Forest classifier and train it.
# classifier_RF = ee.Classifier.smileRandomForest(50)

# # Train the classifier.
# trained_RF = classifier_RF.train(training, 'class', bands)

# # Classify the input imagery.
# classified_RF = image.select(bands).classify(trained_RF)


#### Assess accuracy

In [19]:
#-----------accuracy for model 1: SVM-----------#
# Get a confusion matrix representing resubstitution accuracy.
trainAccuracy_SVM = trained_SVM.confusionMatrix()
print('SVM Resubstitution error matrix: ', trainAccuracy_SVM.getInfo())
print('SVM Training overall accuracy: ', trainAccuracy_SVM.accuracy())
# Validation
validated_SVM = validation.classify(trained_SVM)
testAccuracy_SVM = validated_SVM.errorMatrix('class', 'classification')
print('SVM Validation error matrix: ', testAccuracy_SVM)
print('SVM Validation overall accuracy: ', testAccuracy_SVM.accuracy())

# #-----------accuracy for 2: CART-----------#
# # Get a confusion matrix representing resubstitution accuracy.
# trainAccuracy_CART = trained_CART.confusionMatrix()
# print('CART Resubstitution error matrix: ', trainAccuracy_CART)
# print('CART Training overall accuracy: ', trainAccuracy_CART.accuracy())
# # Validation
# validated_CART = validation.classify(trained_CART)
# testAccuracy_CART = validated_CART.errorMatrix('class', 'classification')
# print('CART Validation error matrix: ', testAccuracy_CART)
# print('CART Validation overall accuracy: ', testAccuracy_CART.accuracy())

# #-----------accuracy for model 3: RF-----------#
# # Get a confusion matrix representing resubstitution accuracy.
# trainAccuracy_RF = trained_RF.confusionMatrix()
# print('RF Resubstitution error matrix: ', trainAccuracy_RF)
# print('RF Training overall accuracy: ', trainAccuracy_RF.accuracy())
# # Validation
# validated_RF = validation.classify(trained_RF)
# testAccuracy_RF = validated_RF.errorMatrix('class', 'classification')
# print('RF Validation error matrix: ', testAccuracy_RF)
# print('RF Validation overall accuracy: ', testAccuracy_RF.accuracy())


EEException: Property 'B3' of feature '1_00000000000000000000_0' is missing.